In [3]:
import numpy as np
import random

In [1]:
from kdtree import *

import open3d as o3d
from pyntcloud import PyntCloud

In [2]:
def feature_extraction_fpfh(cloud_numpy, voxel_size = 0.05):
    point_cloud_o3d = o3d.geometry.PointCloud()
    point_cloud_o3d.points = o3d.utility.Vector3dVector(cloud_numpy[:,0:3])
    
    point_cloud_o3d = point_cloud_o3d.voxel_down_sample(voxel_size) 
    
    radius_normal = voxel_size * 2 
    point_cloud_o3d.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    point_cloud_o3d_fpfh = o3d.registration.compute_fpfh_feature(
        point_cloud_o3d,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return point_cloud_o3d, point_cloud_o3d_fpfh

In [4]:
def feature_based_icp(cloud_1, cloud_2, voxel_size = 0.05):
    source_cloud, source_feature = feature_extraction_fpfh(cloud_2, voxel_size)
    target_cloud, target_feature = feature_extraction_fpfh(cloud_1, voxel_size)
    
    distance_threshold = voxel_size * 1.5 
    result = o3d.registration.registration_fast_based_on_feature_matching(
        source_cloud, target_cloud, source_feature, target_feature, o3d.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [5]:
def load_cloud(path):
    point_cloud_pynt = PyntCloud.from_file(path)
    point_cloud_o3d = point_cloud_pynt.to_instance("open3d", mesh=False)
    cloud = np.asarray(point_cloud_o3d.points)[:,0:3]
    return cloud

def load_cloud_and_color(path):
    point_cloud_pynt = PyntCloud.from_file(path)
    point_cloud_o3d = point_cloud_pynt.to_instance("open3d", mesh=False)
    cloud = np.asarray(point_cloud_o3d.points)[:,0:3]
    return cloud, point_cloud_o3d.colors

def Transform_cloud(cloud, R, t):
    new_cloud = np.dot(R, np.transpose(cloud)) + np.reshape(t, [3,1])
    return new_cloud.transpose()

In [6]:
# randomly sample point in cloud second
def find_match(root_cloud_first, points_first, cloud_second, ratio = 0.1, distance_threshold = 1.0):
    #root_cloud_first = kdtree_construction(points_first, leaf_size=4)
    matched_first = []
    matched_second = []
    k = 1
    
    #distance_threshold = 1.0
    subsample_ids = np.random.choice(cloud_second.shape[0], int(cloud_second.shape[0]*ratio), replace=False)
    distance_sum = 0
    for idx in subsample_ids:
        query = np.asarray(cloud_second[idx,:])
        result_set = KNNResultSet(capacity=k)
        kdtree_knn_search(root_cloud_first, points_first, result_set, query)
        if(result_set.dist_index_list[0].distance > distance_threshold):
            continue
        match_id = result_set.dist_index_list[0].index
        matched_first.append(points_first[match_id,:])
        matched_second.append(query)
        #print(result_set.dist_index_list[0].distance)
        distance_sum += result_set.dist_index_list[0].distance
        
    distance_sum = distance_sum / len(matched_first)
    return np.asarray(matched_first), np.asarray(matched_second), distance_sum

def SvdBasedPoseEstimation(matched_1, matched_2):
    centroid_1 = matched_1.mean(axis=0)
    centroid_2 = matched_2.mean(axis=0)
    
    centered_1 = matched_1 - centroid_1
    centered_2 = matched_2 - centroid_2
    
    Matrix_W = np.dot(np.transpose(centered_1), centered_2)
    U,eigenvalues,VT = np.linalg.svd(Matrix_W)
    #eigenvectors = np.transpose(VT)
    R = np.dot(U, VT)
    t = np.transpose(centroid_1) - np.dot(R, np.transpose(centroid_2))
    return R, t

def ICP(points_first, points_second, iteration = 10, ratio = 0.1, max_distance = 1.0, verbose = True):    
    result_R = np.eye(3)
    result_t = np.zeros(3)
    points_second_transformed = Transform_cloud(points_second[:,0:3], result_R, result_t)
    root_cloud_first = kdtree_construction(points_first, leaf_size=4)
    for i in range(iteration):
        if(verbose):
            print('--> iteration : ', i)
            
        if(i > iteration*2/3):
            max_distance_t = max_distance/3
        elif(i > iteration/2):
            max_distance_t = max_distance/2
        else:
            max_distance_t = max_distance
        matched_first, matched_second, distance_average = find_match(root_cloud_first, points_first[:,0:3], points_second_transformed, ratio, max_distance_t)
        if(verbose):
            print('    matches : ', matched_first.shape[0], ' average distance : ',distance_average)
        if(distance_average < 0.05):
            if(verbose):
                print(' Converged !')
            break
        R, t = SvdBasedPoseEstimation(matched_first, matched_second)
        result_R = np.dot(R, result_R)
        result_t = np.dot(R, result_t) + t
        points_second_transformed = Transform_cloud(points_second_transformed, R, t)
        
    return points_second_transformed, result_R, result_t

In [7]:
def inverse_T(T):
    Final_T = np.eye(4,4)
    inverse_R = np.transpose(T[0:3, 0:3])
    inverse_t = -np.dot(inverse_R, T[0:3, 3])
    Final_T[0:3, 0:3] = inverse_R
    Final_T[0:3, 3] = inverse_t
    return Final_T

def rotationMatrixToEulerAngles(R):
    sy = np.sqrt(R[0,0] * R[0,0] +R[1,0] * R[1,0] );
    singular = sy < 1e-6;
    if (not singular) :
        x = np.arctan2(R[2,1] , R[2,2]);
        y = np.arctan2(-R[2,0], sy);
        z = np.arctan2(R[1,0], R[0,0]);
    else :
        x = np.arctan2(-R[1,2], R[1,1]);
        y = np.arctan2(-R[2,0], sy);
        z = 0;
    #if 1
    x = x*180.0/3.141592653589793;
    y = y*180.0/3.141592653589793;
    z = z*180.0/3.141592653589793;
    #endif
    return np.array([x, y, z]);

In [8]:
def FPFH_and_ICP(cloud_1, cloud_2, iteration = 10, verbose = True):
    FPFH_result = feature_based_icp(cloud_1, cloud_2, 0.2)
    result_R = FPFH_result.transformation[0:3, 0:3]
    result_t = FPFH_result.transformation[0:3, 3]
    points_second_transformed = Transform_cloud(cloud_2, result_R, result_t)
    points_second_transformed, R, t = ICP(cloud_1, points_second_transformed, iteration, 0.1, 3.0, verbose)
    result_R = np.dot(R, result_R)
    result_t = np.dot(R, result_t) + t
    return points_second_transformed, result_R, result_t

#### Initialize with human

In [9]:
human_guess = np.array([
[1.651528358459, -0.080790385604, 3.463548421860, -0.315829426050],
[-3.460947036743, -0.211976200342, 1.645343184471, 13.699180603027],
[0.156660273671, -3.831290006638, -0.164068713784, 2.240842103958],
])
Rs = human_guess[0:3, 0:3]
scale_guess = np.sqrt(np.dot(np.transpose(Rs), Rs)[0,0])

result_R = human_guess[0:3, 0:3] / scale_guess
result_t = human_guess[0:3, 3]
print("the initial scale guess : ", scale_guess)

the initial scale guess :  3.837999837321662


In [17]:
scale_guess = 3.85

In [18]:
points_first = load_cloud("D:/SFM/winter_garden/garden_l_sub_1.ply")
points_second, colors_second = load_cloud_and_color("D:/SFM/CloudMatch/715_garden/sparse_cloud.ply")
points_second = points_second * scale_guess
points_second_transformed = Transform_cloud(points_second, result_R, result_t)

In [19]:
points_second_transformed, R, t = ICP(points_first, points_second_transformed, 30, 0.01, 3.0, True)

--> iteration :  0
    matches :  1728  average distance :  0.5285210585789266
--> iteration :  1
    matches :  1733  average distance :  0.42834294525328
--> iteration :  2
    matches :  1722  average distance :  0.36708817992997017
--> iteration :  3
    matches :  1728  average distance :  0.3133070564035144
--> iteration :  4
    matches :  1710  average distance :  0.28176490966817597
--> iteration :  5
    matches :  1729  average distance :  0.25561617308014917
--> iteration :  6
    matches :  1720  average distance :  0.24897081569412663
--> iteration :  7
    matches :  1726  average distance :  0.23657414288781903
--> iteration :  8
    matches :  1719  average distance :  0.23671343877533346
--> iteration :  9
    matches :  1727  average distance :  0.2160523843225688
--> iteration :  10
    matches :  1722  average distance :  0.21244436966571306
--> iteration :  11
    matches :  1719  average distance :  0.22439926139860117
--> iteration :  12
    matches :  1723  ave

In [20]:
Final_R = np.dot(R, result_R)
Final_t = np.dot(R, result_t) + t
Final_T = np.eye(4,4)
Final_T[0:3, 0:3] = Final_R
Final_T[0:3, 3] = Final_t
print("Final Matrix is : ")
print(Final_T)
print("Inverse T is : \n", inverse_T(Final_T))
print("Euler angles are : ", rotationMatrixToEulerAngles(inverse_T(Final_T)[0:3, 0:3]))
print("Should copy the Inverse Final matrix to unity.")
print("the initial scale guess : ", scale_guess)

Final Matrix is : 
[[ 0.44141694 -0.10947157  0.8905992  -0.26597446]
 [-0.89696069 -0.02645493  0.44131808 13.53898264]
 [-0.02475103 -0.99363786 -0.10986932  1.46913966]
 [ 0.          0.          0.          1.        ]]
Inverse T is : 
 [[ 0.44141694 -0.89696069 -0.02475103 12.29770352]
 [-0.10947157 -0.02645493 -0.99363786  1.78884905]
 [ 0.8905992   0.44131808 -0.10986932 -5.57670775]
 [ 0.          0.          0.          1.        ]]
Euler angles are :  [103.97999772 -62.94864593 -13.92836274]
Should copy the Inverse Final matrix to unity.
the initial scale guess :  3.85


In [21]:
point_cloud_o3d_1 = o3d.geometry.PointCloud()
point_cloud_o3d_1.points = o3d.utility.Vector3dVector(points_first[:,0:3])
colors = [[1, 0, 0] for i in range(points_first.shape[0])]
point_cloud_o3d_1.colors = o3d.utility.Vector3dVector(colors)

point_cloud_o3d_2 = o3d.geometry.PointCloud()
points_second_transformed = Transform_cloud(points_second, Final_R, Final_t)
point_cloud_o3d_2.points = o3d.utility.Vector3dVector(points_second_transformed[:,0:3])
point_cloud_o3d_2.colors = o3d.utility.Vector3dVector(np.asarray(colors_second) / 255)

o3d.visualization.draw_geometries([point_cloud_o3d_1, point_cloud_o3d_2])

![image](images/fused_5_matched.PNG)

In [34]:
# write result to txt
def print_final_result_for_copy(T):
    for i in range(4):
        print(str(T[i,0])+'f, ' +str(T[i,1])+'f, '+str(T[i,2])+'f, '+str(T[i,3])+'f,')
    
print_final_result_for_copy( inverse_T(Final_T))

0.44148853065346166f, -0.8969043435743231f, -0.025504426654194855f, 12.310358951183794f,
-0.10950857000724631f, -0.02564840744604299f, -0.993654932353477f, 1.7681489351641213f,
0.8905591595082349f, 0.4414801681147646f, -0.10954216313511526f, -5.588568650747423f,
0.0f, 0.0f, 0.0f, 1.0f,
